In [1]:
# === 1. Minimal Package Installation ===
# ✅ Minimal working environment for local LLM + embeddings + RAG
!pip install --quiet \
    langchain-community==0.2.0 \
    sentence-transformers==2.6.1 \
    faiss-cpu \
    transformers==4.37.2 \
    accelerate==0.29.3 \
    pypdf



# === 2. Stable Imports ===
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_community.llms import HuggingFacePipeline
from langchain.chains import RetrievalQA
from transformers import pipeline
import os

# === 3. Config ===
PDF_PATH = "/kaggle/input/comapny-documents/companydoc.pdf"  # Verify this path exists
VECTOR_STORE_DIR = "vectorstore"
EMBEDDING_MODEL = "all-MiniLM-L6-v2"
LLM_MODEL = "google/flan-t5-small"

# === 4. Document Processing ===
def load_documents():
    loader = PyPDFLoader(PDF_PATH)
    docs = loader.load()
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=500,
        chunk_overlap=100,
        separators=["\n\n", "\n", ".", " "]
    )
    return splitter.split_documents(docs)

# === 5. Vector Store ===
def create_vectorstore(chunks):
    embeddings = HuggingFaceEmbeddings(model_name=EMBEDDING_MODEL)
    vectorstore = FAISS.from_documents(chunks, embeddings)
    vectorstore.save_local(VECTOR_STORE_DIR)
    return vectorstore

# === 6. LLM Setup ===
def get_local_llm():
    os.environ['TOKENIZERS_PARALLELISM'] = 'false'  # Prevent fork warnings
    local_pipeline = pipeline(
        task="text2text-generation",
        model=LLM_MODEL,
        tokenizer=LLM_MODEL,
        max_new_tokens=300,
        device=-1  # CPU
    )
    return HuggingFacePipeline(pipeline=local_pipeline)

# === 7. RAG Chain ===
def create_rag_chain(vectorstore, llm):
    return RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",
        retriever=vectorstore.as_retriever(),
        return_source_documents=True
    )

# === 8. Execution ===
def run_kaggle_app(query):
    if os.path.exists(VECTOR_STORE_DIR):
        vectorstore = FAISS.load_local(
            VECTOR_STORE_DIR,
            HuggingFaceEmbeddings(model_name=EMBEDDING_MODEL),
            allow_dangerous_deserialization=True
        )
    else:
        vectorstore = create_vectorstore(load_documents())
    
    result = create_rag_chain(vectorstore, get_local_llm())({"query": query})
    print(f"Answer: {result['result']}\nSources: {[d.metadata['source'] for d in result['source_documents']]}")

# === 9. Run ===
run_kaggle_app("Summarize the document in simple words.")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.4/129.4 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 43.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.3/163.3 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 49.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 26.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 36.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 397.1/397.1 kB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.8/311.8 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 70.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13

2025-08-01 03:43:44.452890: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1754019824.723668      13 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1754019824.800176      13 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

/tmp/ipykernel_13/1421017056.py:51: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 0.3. An updated version of the class exists in the from rom langchain-huggingface package and should be used instead. To use it run `pip install -U from rom langchain-huggingface` and import as `from from rom langchain_huggingface import llms import HuggingFacePipeline`.
  return HuggingFacePipeline(pipeline=local_pipeline)
/tmp/ipykernel_13/1421017056.py:73: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use invoke instead.
  result = create_rag_chain(vectorstore, get_local_llm())({"query": query})


Answer:                                                                                                                                                      
Sources: ['/kaggle/input/comapny-documents/companydoc.pdf', '/kaggle/input/comapny-documents/companydoc.pdf', '/kaggle/input/comapny-documents/companydoc.pdf', '/kaggle/input/comapny-documents/companydoc.pdf']
